In [89]:
! pip install backtrader

     -------------------------------------- 410.1/410.1 KB 3.7 MB/s eta 0:00:00


In [527]:
import math 
import numpy as np 
import pandas as pd 
from datetime import date, timedelta, datetime
from pandas.plotting import register_matplotlib_converters
import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
from sklearn.metrics import mean_absolute_error, mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import RobustScaler
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix
import datetime
import backtrader as bt
import backtrader.analyzers as btanalyzers

In [528]:
import warnings
warnings.filterwarnings('ignore')

# Step #1 Load the Time Series Data

In [529]:
def read_data(n):
    if n == 1:
        df = pd.read_csv(r'E:\IAQF\data updated\d1.csv',index_col = 0)
    elif n == 2:
        df = pd.read_csv(r'E:\IAQF\data updated\d2.csv',index_col = 0)
    elif n == 3:
        df = pd.read_csv(r'E:\IAQF\data updated\d3.csv',index_col = 0)
    elif n == 4:
        df = pd.read_csv(r'E:\IAQF\data updated\d4.csv',index_col = 0)
    elif n == 5:
        df = pd.read_csv(r'E:\IAQF\data updated\d5.csv',index_col = 0)
    elif n == 6:
        df = pd.read_csv(r'E:\IAQF\data updated\data_gmm_1_2003.csv',index_col = 0)
    elif n == 7:
        df = pd.read_csv(r'E:\IAQF\data updated\data_hmm_1_2003.csv',index_col = 0)
    df.index = pd.to_datetime(df.index)
    df = df.loc[:,['Open', 'High', 'Low', 'Close','momentum_rsi','trend_macd','volatility_dcp','volatility_dcp','momentum_ppo','trend_cci','T10YIE', 'state']]
    #df = df.loc[:,[ 'Close','momentum_rsi','trend_macd','volatility_dcp','volatility_dcp','momentum_ppo','trend_cci','T10YIE', 'state']]
    #df = df.loc[:,['Open', 'High', 'Low', 'Close','momentum_rsi', 'volatility_bbp', 'volatility_dcp', 'momentum_ppo', 'momentum_tsi', 'trend_cci', 'momentum_ppo_signal', 'momentum_roc', 'trend_macd', 'trend_macd_signal', 'trend_adx_pos', 'momentum_stoch', 'trend_trix', 'trend_vortex_ind_diff', 'momentum_wr', 'trend_vortex_ind_neg', 'volatility_kcp', 'trend_adx_neg', 'trend_kst_sig', 'momentum_ao', 'volatility_ui', 'trend_kst', 'volatility_dcw', 'volatility_kcw', 'trend_vortex_ind_pos','state']]
    df['state'] = df['state'].map({-1: 0, 0: 1,1: 2})
    #print('value_counts for state:\n', df.state.value_counts())
    #print('\n')
    return df

# Step #4 Train-Test Split

In [530]:
def train_test_split(df):
    train = df.loc[:'2017-12-10',:]
    test = df.loc['2017-12-11':, :]
    return train, test

def check_length(train,test,df):
    print('value_counts for training state:\n',train.state.value_counts())
    print('\n')
    print('value_counts for testing state:\n',test.state.value_counts())
    print('\n')
    print('length of train data:',len(train), '\nlength of test data:',len(test))
    print('\n')
    print('proportion of train data:',len(train)/len(df),'\nproportion of test data:',len(test)/len(df))
    print('\n')

# Step #5 Scaling and Transforming the Data

In [531]:
def scale_transform_data(train, test):
    f_columns = df.columns[:-1]
    f_transformer = RobustScaler()
    f_transformer = f_transformer.fit(train[f_columns].to_numpy())
    train.loc[:, f_columns] = f_transformer.transform(train[f_columns].to_numpy())
    test.loc[:, f_columns] = f_transformer.transform(test[f_columns].to_numpy())
    return train,test

# Step #6 Cuting time series into sub sequencies & oneHot encoder  for y

In [532]:
#reshape to [#samples, #time_steps, #n_features]
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps),:-1].values 
        Xs.append(v)        
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys).reshape(-1, 1)

In [533]:
def onehotencoder_y(y_train,y_test):
    enc = OneHotEncoder(handle_unknown='ignore')
    enc.fit(y_train)
    y_train = enc.transform(y_train).toarray()
    y_test = enc.transform(y_test).toarray()
    return y_train, y_test, enc

# Step #7 Build LSTM & Bidirectional LSTM Model on Train data in Keras

In [534]:
def weights(train_data):
    class_weights = class_weight.compute_class_weight('balanced',np.unique(train_data.iloc[:,-1]),(train_data.iloc[:,-1]))
    class_weights = dict(enumerate(class_weights))
    return class_weights

In [535]:
def Bi_LSTM_model(X_train,y_train):
    model = keras.Sequential()
    model.add(
    keras.layers.Bidirectional(
        keras.layers.LSTM(units=128,input_shape=[X_train.shape[1], X_train.shape[2]])))
    #model.add(keras.layers.Dropout(rate=0.3))
    model.add(keras.layers.Dense(units=64, activation='relu'))
    model.add(keras.layers.Dense(y_train.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [536]:
def LSTM_model(X_train,y_train):
    model = keras.Sequential()
    model.add(LSTM(units=64,input_shape=[X_train.shape[1], X_train.shape[2]]))
    model.add(keras.layers.Dropout(rate=0.5))
    #model.add(keras.layers.Dense(units=64, activation='relu'))
    #model.add(keras.layers.Dropout(rate=0.5))
    model.add(keras.layers.Dense(units=32, activation='relu'))
    model.add(keras.layers.Dropout(rate=0.5))
    model.add(keras.layers.Dense(y_train.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [537]:
def fit_LSTM(weight,X_train,y_train,train_data,model):
    if weight == 1:
        history = model.fit(
            X_train, y_train,
            epochs=15,
            batch_size=32,
            validation_split=0.1,
            shuffle=False,
            verbose=0,
            class_weight=weights(train_data))
    elif weight == 0:
        history = model.fit(
            X_train, y_train,
            epochs=15,
            batch_size=32,
            validation_split=0.1,
            verbose = 0,
            shuffle=False)
    return history

# Step #8 Fit Bidirectional LSTM Model on Test data in Keras

In [538]:
def pred(model,X_test):
    y_pred = model.predict(X_test)
    return y_pred

In [539]:
def compute_final_df(y_train_noenc,y_test_noenc,y_pred,test):
    enc = onehotencoder_y(y_train_noenc,y_test_noenc)[2]
    a = enc.inverse_transform(y_pred)
    b = enc.inverse_transform(y_pred).shape[0]
    y_pred2 = np.reshape(a, (1, b))[0][4:]
    final_pred_df = test.loc['2018':,:]
    final_pred_df.insert(loc=test.shape[1],column='State',value=y_pred2)
    final_pred_df = final_pred_df.drop('state', 1)
    final_pred_df['State'] = final_pred_df['State'].map({0: -1, 1: 0,2: 1})
    final_pred_df = final_pred_df.reset_index()
    final_y_pred = final_pred_df.loc[:,'State']
    
    test['state'] = test['state'].map({0: -1, 1: 0,2: 1})
    final_y_test = test.loc['2018':,'state']
    return final_y_test,final_y_pred,final_pred_df

# Step #9 Evaluate

In [540]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    import itertools
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [541]:
def evaluate(history,X_train, y_train, y_train_noenc,y_test_noenc,X_test,y_test,y_pred,final_y_test,final_y_pred):
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='validation')
    plt.legend();
    print('\n')
    
    X_train =X_train[4:]
    y_train = y_train[4:]
    X_test = X_test[4:]
    y_test = y_test[4:]
    y_pred = y_pred[4:]
    
    scores1 = model.evaluate(X_train, y_train, verbose=0)
    print('Accuracy on training data: {}% \n Error on training data: {}'.format(scores1[1], 1 - scores1[1]))   

    scores2 = model.evaluate(X_test, y_test, verbose=0)
    print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1]))
    print('\n')
    
    print('model_evaluate:',model.evaluate(X_test, y_test))
    print('\n')
    
    cnf_matrix = confusion_matrix(final_y_test, final_y_pred,labels=[-1,0,1])
    np.set_printoptions(precision=2)
    # Plot non-normalized confusion matrix
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=['Bear', 'Static', 'Bull'],title='Confusion matrix')
    print('\n')

# Step #10 Compute CSV

In [542]:
def output(n,weight,final_test):
    if n == 1 and weight == 1:
        final_test.to_csv('E:/IAQF/result NN/d1_adjust.csv')
    elif n == 1 and weight == 0:
        final_test.to_csv('E:/IAQF/result NN/d1_without_adjust.csv')
    elif n == 2 and weight == 1:
        final_test.to_csv('E:/IAQF/result NN/d2_adjust.csv')
    elif n == 2 and weight == 0:
        final_test.to_csv('E:/IAQF/result NN/d2_without_adjust.csv')
    elif n == 3 and weight == 1:
        final_test.to_csv('E:/IAQF/result NN/d3_adjust.csv')
    elif n == 3 and weight == 0:
        final_test.to_csv('E:/IAQF/result NN/d3_without_adjust.csv')
    elif n == 4 and weight == 1:
        final_test.to_csv('E:/IAQF/result NN/d4_adjust.csv')
    elif n == 4 and weight == 0:
        final_test.to_csv('E:/IAQF/result NN/d4_without_adjust.csv')
    elif n == 5 and weight == 1:
        final_test.to_csv('E:/IAQF/result NN/d5_adjust.csv')
    elif n == 5 and weight == 0:
        final_test.to_csv('E:/IAQF/result NN/d5_without_adjust.csv')

# BackTesting

In [543]:
class TestPositions(bt.Strategy):
    def __init__(self):
        pass

    def log(self, txt, dt=None):
        """ Logging function fot this strategy"""
        dt = dt or self.data.datetime[0]
        if isinstance(dt, float):
            dt = bt.num2date(dt)
        print("%s, %s" % (dt, txt))

    def print_signal(self):
        self.log(
            f"o {self.datas[0].open[0]:7.2f} "
            f"h {self.datas[0].high[0]:7.2f} "
            f"l {self.datas[0].low[0]:7.2f} "
            f"c {self.datas[0].close[0]:7.2f} "
            f"v {self.datas[0].volume[0]:7.0f} "
        )

    def next(self):
        self.print_signal()
        

In [544]:
from backtrader.feeds import PandasData
class PandasData(PandasData):
    '''
    The ``dataname`` parameter inherited from ``feed.DataBase`` is the pandas
    DataFrame
    '''
    lines = ('state',)
    params = (
        ('datetime', 0),
        ('high', 'High'),
        ('low', 'Low'),
        ('close', 'Close'),
        ('volume', None),
        ('openinterest', None),
        ('state', 5),
    )
    datafields = PandasData.datafields + (['state'])

In [545]:
class MarketStatus(bt.Strategy): 

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        #print(f'{dt.isoformat()} {txt}') # Comment this line when running optimization

    def __init__(self):
        self.dataclose = self.datas[0].close

        # Order variable will contain ongoing order details/status
        self.order = None

        # Instantiate market status
        self.state = self.datas[0].state

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # An active Buy/Sell order has been submitted/accepted - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, {order.executed.price:.2f}')
            elif order.issell():
                self.log(f'SELL EXECUTED, {order.executed.price:.2f}')
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Reset orders
        self.order = None

    def next(self):

        # Check for open orders
        if self.order:
            return
        
        if self.state[0] == 1:
            if not self.position:
                #print('no position',self.state[0], self.position.size)
                self.log(f'BUY CREATE {self.dataclose[0]:2f} {self.state[0]:2f}')
                self.order = self.buy()
            elif self.position.size < 0: # already have a sell order
                #print('sell position',self.state[0], self.position.size)
                self.order = self.close()
                self.log(f'CLOSE CREATE {self.dataclose[0]:2f}')
                self.log(f'BUY CREATE {self.dataclose[0]:2f} {self.state[0]:2f}')
                self.order = self.buy()
            elif self.position.size > 0: # already have a buy order
                # pass
                # self.log(f'BUY CREATE {self.dataclose[0]:2f} {self.state[0]:2f}')
                # self.order = self.buy()

                if len(self) == (self.bar_executed + 5):
                    #print('buy position, 5 days',self.state[0], self.position.size)
                    # self.order = self.close()
                    # self.log(f'CLOSE CREATE {self.dataclose[0]:2f}')
                    self.log(f'BUY CREATE {self.dataclose[0]:2f} {self.state[0]:2f}')
                    self.order = self.buy()
                    self.log('Chase')
                elif len(self) >= (self.bar_executed + 15):
                    self.order = self.close()
                    self.log(f'CLOSE CREATE {self.dataclose[0]:2f}')
                else:
                    #print('buy position, less than 5 days',self.state[0], self.position.size)
                    pass
            else:
                #print('error',self.state[0], self.position.size)
                pass
        
        elif self.state[0] == -1:
            if not self.position:
                #print('no position',self.state[0], self.position.size)                
                self.log(f'SELL CREATE {self.dataclose[0]:2f} {self.state[0]:2f}')
                self.order = self.sell()
            elif self.position.size > 0: # already have a buy order
                #print('sell position',self.state[0], self.position.size)                
                self.order = self.close()
                self.log(f'CLOSE CREATE {self.dataclose[0]:2f}')
                self.log(f'SELL CREATE {self.dataclose[0]:2f} {self.state[0]:2f}')
                self.order = self.sell()
            elif self.position.size < 0: # already have a sell order
                # pass
                # Chase every order
                # self.log(f'SELL CREATE {self.dataclose[0]:2f} {self.state[0]:2f}')
                # self.order = self.sell()

                # Chase 10 
                if len(self) == (self.bar_executed + 5):
                    #print('buy position, 5 days',self.state[0], self.position.size)
                    # self.order = self.close()
                    # self.log(f'CLOSE CREATE {self.dataclose[0]:2f}')
                    self.log(f'SELL CREATE {self.dataclose[0]:2f} {self.state[0]:2f}')
                    self.order = self.sell()
                    # self.log('Chase')
                elif len(self) >= (self.bar_executed + 15):
                    self.order = self.close()
                    self.log(f'CLOSE CREATE {self.dataclose[0]:2f}')
                else:
                    #print('buy position, less than 5 days',self.state[0], self.position.size)
                    pass

            else:
                #print('error',self.state[0], self.position.size)
                pass
        
        else:
            if not self.position:
                #print('0, nothing',self.state[0], self.position.size)
                pass
            else:
                self.log(f'Hold {self.dataclose[0]:2f} {self.state[0]:2f}')
            
        # elif self.state[0] == -1:
        #     if not self.position:
        #         pass
        #     else:
        #         self.order = self.close()
        #         self.log(f'CLOSE CREATE {self.dataclose[0]:2f}')
        #     self.log(f'SELL CREATE {self.dataclose[0]:2f} {self.state[0]:2f}')
        #     self.order = self.sell()
                
        # else:
        #     if not self.position:
        #         pass
        #     else:
        #         self.log(f'Hold {self.dataclose[0]:2f} {self.state[0]:2f}')
        # print(self.position)


            # if len(self) >= (self.bar_executed + 5):
            #     self.log(f'CLOSE CREATE {self.dataclose[0]:2f}')
            #     self.order = self.close()

# Operation Model

In [546]:
df = read_data(1)
train = train_test_split(df)[0]
test = train_test_split(df)[1]
test

,Open,High,Low,Close,momentum_rsi,trend_macd,volatility_dcp,volatility_dcp,momentum_ppo,trend_cci,T10YIE,state
Date,,,,,,,,,,,,
2017-12-11,1569.719971,1573.699951,1569.719971,1573.540039,71.355599,11.798182,0.899957,0.899957,0.762609,110.376969,1.88,1
2017-12-12,1573.530029,1578.609985,1573.530029,1574.920044,71.963060,12.203902,0.920171,0.920171,0.787784,113.208722,1.89,1
2017-12-13,1575.449951,1580.430054,1574.819946,1574.819946,71.844047,12.374713,0.917897,0.917897,0.797831,109.348774,1.88,1
2017-12-14,1575.160034,1577.849976,1567.260010,1567.260010,63.325846,11.764445,0.789380,0.789380,0.757898,78.428810,1.87,1
2017-12-15,1568.900024,1584.780029,1568.900024,1582.079956,70.667844,12.334466,0.953075,0.953075,0.793491,111.107341,1.88,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-23,2750.840088,2776.620117,2750.840088,2768.360107,55.721697,1.598689,0.941414,0.941414,0.058593,121.819263,2.47,1
2021-12-27,2769.340088,2803.879883,2769.340088,2803.739990,59.777151,7.358879,0.999169,0.999169,0.269159,164.907229,2.50,1
2021-12-28,2803.899902,2813.590088,2795.709961,2798.350098,58.892209,11.358024,0.914363,0.914363,0.414709,163.360053,2.50,1


In [547]:
def operation():
    for n in range(1,8):
        for weight in range(0,2):
            df = read_data(n)

            train = train_test_split(df)[0]
            test = train_test_split(df)[1]

            #check_length(train,test,df)

            train = scale_transform_data(train, test)[0]
            test = scale_transform_data(train, test)[1]

            time_steps = 10 # Use how many days x to predict next y
            X_train, y_train_noenc = create_dataset(train, train.state, time_steps)
            X_test, y_test_noenc = create_dataset(test, test.state, time_steps)
            #print('X_train.shape:',X_train.shape)
            #print('X_test.shape:',X_test.shape)

            # oneHot Encoder for y
            y_train = onehotencoder_y(y_train_noenc,y_test_noenc)[0]
            y_test = onehotencoder_y(y_train_noenc,y_test_noenc)[1]
            #print('y_train.shape:',y_train.shape, '\ny_test.shape:',y_test.shape)
            
            model = LSTM_model(X_train,y_train)
            history = fit_LSTM(weight,X_train,y_train,train,model)
            y_pred = pred(model,X_test)
            
            final_y_test = compute_final_df(y_train_noenc,y_test_noenc,y_pred,test)[0]
            final_y_pred = compute_final_df(y_train_noenc,y_test_noenc,y_pred,test)[1]
            #evaluate(history,X_train, y_train, y_train_noenc,y_test_noenc,X_test,y_test,y_pred,final_y_test,final_y_pred)
            
            final_pred_df = compute_final_df(y_train_noenc,y_test_noenc,y_pred,test)[2]
            #output(n,weight,final_pred_df)
            final_pred_df.State.value_counts()
            
            dataf = final_pred_df
            dataf['Date']=pd.to_datetime(dataf['Date'])
            
            cerebro = bt.Cerebro()

            num = dataf.shape[1]-1
            data = PandasData(dataname = dataf,fromdate=datetime.datetime(2018, 1, 2),todate=datetime.datetime(2021, 12, 30),datetime = 0,open = 1,high = 2,low = 3, close = 4,state = num)

            # Add data to Cerebro
            cerebro.adddata(data)

            # Add strategy to Cerebro
            cerebro.addstrategy(MarketStatus)

            # Default position size
            cerebro.addsizer(bt.sizers.SizerFix, stake=1)

            # Add analytics to Cerebro
            cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='SharpeRatio')
            cerebro.addanalyzer(btanalyzers.AnnualReturn, _name='AnnualReturn')
            cerebro.addanalyzer(btanalyzers.DrawDown, _name='DrawDown')
            # cerebro.addanalyzer(btanalyzers.TimeDrawDown, _name='TimeDrawDown')
            cerebro.addanalyzer(btanalyzers.PositionsValue, _name='PositionsValue')
            cerebro.addanalyzer(btanalyzers.LogReturnsRolling, _name='LogReturnsRolling')
            cerebro.addanalyzer(btanalyzers.PeriodStats, _name='PeriodStats')
            cerebro.addanalyzer(btanalyzers.Returns, _name='Returns')
            cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name='TradeAnalyzer')
            cerebro.addanalyzer(btanalyzers.Transactions, _name='Transactions')

            if __name__ == '__main__':
                # Run Cerebro Engine
                start_portfolio_value = cerebro.broker.getvalue()

                # cerebro.run()
                thestrats = cerebro.run()
                thestrat = thestrats[0]

                end_portfolio_value = cerebro.broker.getvalue()
                pnl = end_portfolio_value - start_portfolio_value

                #print(f'Starting Portfolio Value: {start_portfolio_value:2f}')
                #print(f'Final Portfolio Value: {end_portfolio_value:2f}')
                print('n=',n,'weight=',weight,f'PnL= {pnl:.2f}\n')
            
            #if n == 2 and weight ==1:
                #return final_pred_df

In [548]:
operation()

n= 1 weight= 0 PnL= 0.00

n= 1 weight= 1 PnL= 2.04

n= 2 weight= 0 PnL= 5.88

n= 2 weight= 1 PnL= 6.94

n= 3 weight= 0 PnL= 0.00

n= 3 weight= 1 PnL= 3.64

n= 4 weight= 0 PnL= 0.00

n= 4 weight= 1 PnL= 9.74

n= 5 weight= 0 PnL= 0.00

n= 5 weight= 1 PnL= -1.53

n= 6 weight= 0 PnL= 38.93

n= 6 weight= 1 PnL= 50.92

n= 7 weight= 0 PnL= 61.83

n= 7 weight= 1 PnL= 52.65



# BackTesting

In [412]:
df = final_pred_df
df['Date']=pd.to_datetime(df['Date'])
df.head()

,Date,momentum_rsi,volatility_bbp,volatility_dcp,momentum_ppo,momentum_tsi,trend_cci,momentum_ppo_signal,momentum_roc,trend_macd,...,volatility_kcp,trend_adx_neg,trend_kst_sig,momentum_ao,volatility_ui,trend_kst,volatility_dcw,volatility_kcw,trend_vortex_ind_pos,State
0,2018-01-02,0.763334,0.392436,0.546789,0.331556,0.972254,0.223726,0.416932,0.184818,0.931015,...,0.362952,-0.645843,0.413976,0.803042,-0.563848,0.387874,-0.727937,-0.873797,0.171424,1
1,2018-01-03,1.009173,0.619581,0.524230,0.373637,1.020410,0.516070,0.417758,0.499525,1.010364,...,1.015932,-0.780194,0.414248,0.828503,-0.563848,0.382116,-0.554112,-0.875997,0.380330,1
2,2018-01-04,1.143704,0.701671,0.459992,0.424942,1.087306,0.723207,0.429618,0.327892,1.107173,...,1.315114,-0.886434,0.414053,0.904297,-0.563850,0.396266,-0.416259,-0.878304,0.618164,1
3,2018-01-05,1.351468,0.879943,0.546823,0.503721,1.188843,0.978949,0.456305,0.333169,1.255571,...,1.870013,-1.004179,0.417039,1.011127,-0.579606,0.429284,-0.295415,-0.871259,1.019058,1
4,2018-01-08,1.405640,0.792764,0.536593,0.568733,1.279728,0.985948,0.491847,0.495910,1.379082,...,1.846394,-1.078818,0.424748,1.171421,-0.579606,0.470972,-0.393115,-0.872691,1.310662,1


# Test 1 Result

In [84]:
# df = df.loc[:,['Open', 'High', 'Low', 'Close','momentum_rsi','trend_macd','volatility_dcp','volatility_dcp','momentum_ppo','trend_cci','T10YIE', 'state']]

# Test 2 Result

In [84]:
# df = df.loc[:,['Open', 'High', 'Low', 'Close','momentum_rsi','trend_macd','volatility_dcp','volatility_dcp','momentum_ppo','trend_cci','T10YIE', 'state']]

# Test 3 Result

In [ ]:
#df = df.loc[:,[ 'Close','momentum_rsi','trend_macd','volatility_dcp','volatility_dcp','momentum_ppo','trend_cci','T10YIE', 'state']]

# Test 4 Result

In [549]:
# df = df.loc[:,['Open', 'High', 'Low', 'Close','momentum_rsi', 'volatility_bbp', 'volatility_dcp', 'momentum_ppo', 'momentum_tsi', 'trend_cci', 'momentum_ppo_signal', 'momentum_roc', 'trend_macd', 'trend_macd_signal', 'trend_adx_pos', 'momentum_stoch', 'trend_trix', 'trend_vortex_ind_diff', 'momentum_wr', 'trend_vortex_ind_neg', 'volatility_kcp', 'trend_adx_neg', 'trend_kst_sig', 'momentum_ao', 'volatility_ui', 'trend_kst', 'volatility_dcw', 'volatility_kcw', 'trend_vortex_ind_pos','state']]

# Test 5 Result